In [90]:
import pandas as pd
import numpy as np

In [91]:
xls = pd.ExcelFile("DA - Task1.xlsx")

In [92]:
xls.sheet_names

['Task', 'Taxonomy']

In [93]:
task_df = pd.read_excel("DA - Task1.xlsx",sheet_name="Task")
taxonomy_df =pd.read_excel("DA - Task1.xlsx",sheet_name="Taxonomy")

In [94]:
task_df.head(3)

,Primary Key,Order Date,Product Category,Complaint,Cause,Correction,Root Cause,Symptom Condition 1,Symptom Component 1,Symptom Condition 2,Symptom Component 2,Symptom Condition 3,Symptom Component 3,Fix Condition 1,Fix Component 1,Fix Condition 2,Fix Component 2,Fix Condition 3,Fix Component 3
0,SO0026296-1,2023-03-08,SPRAYS,VISIBLY NOTICE fasteners under cab on P clips ...,Not tighten at factory.,"GO THROUGH AND RE-TIGHTEN ALL P CLIPS, NUTS, A...",Not Tightened,Loose,Cab P Clip,Loose,Left-Air Duct,Loose,Bulkhead Connector,Retightened,Cab P Clip,Retightened,Left Air Duct,Retightened,Bulkhead Connector
1,SO0026385-1,2023-03-08,SPRAYS,Fuel door will not stay open,GAS STRUT NOT INSTALLED OR ANYWHERE ON MACHINE,FOUND GAS STRUT NOT INSTALLED OR ANYWHERE ON M...,Not Installed,Won't stay open,Fuel Door,NaN,NaN,NaN,NaN,Installed,Gas Strut,NaN,NaN,NaN,NaN
2,SO0026385-11,2023-03-08,SPRAYS,"Compressor pressure line, braided steel, crushed","Compressor pressure line, braided steel, crush...",DRAIN AIR FROM SYSTEM.REMOVE ASSOCIATED P CLIP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
taxonomy_df.head(3)

,Root Cause,Symptom Condition,Symptom Component,Fix Condition,Fix Component
0,Not Tightened,Loose,Cab P Clip,Retightened,Cab P Clip
1,Not Installed,Won't stay open,Fuel Door,Installed,Gas Strut
2,Not Mentioned,Crushed,Compressor Pressure Line,Replaced,Braided Steel


In [96]:
task_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Primary Key          20 non-null     object        
 1   Order Date           20 non-null     datetime64[ns]
 2   Product Category     20 non-null     object        
 3   Complaint            20 non-null     object        
 4   Cause                20 non-null     object        
 5   Correction           20 non-null     object        
 6   Root Cause           2 non-null      object        
 7   Symptom Condition 1  2 non-null      object        
 8   Symptom Component 1  2 non-null      object        
 9   Symptom Condition 2  1 non-null      object        
 10  Symptom Component 2  1 non-null      object        
 11  Symptom Condition 3  1 non-null      object        
 12  Symptom Component 3  1 non-null      object        
 13  Fix Condition 1      2 non-null      

In [111]:
task_df = task_df.iloc[:,:6]

In [112]:
taxonomy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Root Cause          18 non-null     object
 1   Symptom Condition   17 non-null     object
 2   Symptom Component   18 non-null     object
 3   Fix Condition       9 non-null      object
 4   Fix Component       24 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [113]:
# Drop NaN and convert to list
root_causes = taxonomy_df['Root Cause'].dropna().tolist()
symptom_conditions = taxonomy_df['Symptom Condition '].dropna().tolist()
symptom_components = taxonomy_df['Symptom Component'].dropna().tolist()
fix_conditions = taxonomy_df['Fix Condition'].dropna().tolist()
fix_components = taxonomy_df['Fix Component'].dropna().tolist()


In [114]:
from fuzzywuzzy import process

def fuzzy_match(text, choices, threshold>=50):
    if pd.isna(text) or not str(text).strip():
        return ""
    match, score = process.extractOne(str(text), choices)
    return match if score >= threshold else ""


In [115]:
# Tagging Complaint → Symptom
task_df['Symptom_Condition'] = task_df['Complaint'].apply(lambda x: fuzzy_match(x, symptom_conditions))
task_df['Symptom_Component'] = task_df['Complaint'].apply(lambda x: fuzzy_match(x, symptom_components))

# Tagging Cause → Root Cause
task_df['Root_Cause'] = task_df['Cause'].apply(lambda x: fuzzy_match(x, root_causes))

# Tagging Correction → Fix
task_df['Fix_Condition'] = task_df['Correction'].apply(lambda x: fuzzy_match(x, fix_conditions))
task_df['Fix_Component'] = task_df['Correction'].apply(lambda x: fuzzy_match(x, fix_components))


In [116]:
task_df['Symptom_Condition'].isnull().any()

False

In [117]:
task_df['Symptom_Component'].isnull().any()

False

In [118]:
task_df['Fix_Component'].isnull().any()

False

In [119]:
task_df['Fix_Condition'].isnull().any()

False

In [120]:
task_df['Root_Cause'].isnull().any()

False

In [122]:
task_df.to_excel("Tagged_Task1_Output1.xlsx", index=False)